In [17]:
import random
from IPython.display import clear_output

rank_values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, 
               '9':9, 'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}
suits = ['H', 'D', 'C', 'S']

class InteractivePoker:
    def __init__(self):
        self.deck = []
        self.player_hand = []
        self.dealer_hand = []
        self.pot = 0
        self.reset_game()
        
    def reset_game(self):
        self.deck = [r+s for s in suits for r in rank_values]
        random.shuffle(self.deck)
        self.player_hand = [self.deck.pop() for _ in range(5)]
        self.dealer_hand = [self.deck.pop() for _ in range(5)]
        self.pot = 1  # Initial blind
        
    def evaluate_hand(self, hand):
        values = sorted([rank_values[c[0]] for c in hand], reverse=True)
        suits = [c[1] for c in hand]
        
        # Check flush
        flush = len(set(suits)) == 1
        
        # Check straight
        unique = sorted(list(set(values)))
        straight = False
        for i in range(len(unique)-4):
            if unique[i] - unique[i+4] == 4:
                straight = True
                break
        if not straight and {14,2,3,4,5}.issubset(set(values)):
            straight = True
            values = [5,4,3,2,1]  # Treat A-2-3-4-5 as 5-high
            
        # Count duplicates
        count = {}
        for v in values: count[v] = count.get(v, 0) + 1
        sorted_counts = sorted(count.items(), key=lambda x: (-x[1], -x[0]))
        
        if straight and flush: return (8, max(values))
        if any(c[1]==4 for c in sorted_counts): return (7, sorted_counts[0][0])
        if len(sorted_counts)>1 and sorted_counts[0][1]==3 and sorted_counts[1][1]==2:
            return (6, sorted_counts[0][0])
        if flush: return (5, values)
        if straight: return (4, max(values))
        if sorted_counts[0][1] == 3: return (3, sorted_counts[0][0], values)
        if len(sorted_counts)>1 and sorted_counts[0][1]==2 and sorted_counts[1][1]==2:
            return (2, sorted_counts[0][0], sorted_counts[1][0], values)
        if sorted_counts[0][1] == 2: return (1, sorted_counts[0][0], values)
        return (0, values)

    def play_round(self):
        self.reset_game()
        print("Your hand:", self.player_hand)
        
        # Discard phase
        discard = input("Choose cards to discard (space separated): ").upper().split()
        if discard:
            self.player_hand = [c for c in self.player_hand if c not in discard]
            self.player_hand += [self.deck.pop() for _ in range(len(discard))]
            self.pot += 1
        
        
        # Showdown
        player_score = self.evaluate_hand(self.player_hand)
        dealer_score = self.evaluate_hand(self.dealer_hand)
        
        print("\nDealer shows:", self.dealer_hand[0], "...")
        print("Your final hand:", self.player_hand)
        
        if player_score > dealer_score:
            print(f"You win ${self.pot}!")
        else:
            print("Dealer wins!")



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# Define card values and validation
VALID_RANKS = ['2','3','4','5','6','7','8','9','T','J','Q','K','A']
VALID_SUITS = ['H','D','S','C']
rank_values = {r:i+2 for i,r in enumerate(VALID_RANKS)}

class HandValidator:
    @staticmethod
    def is_valid_card(card):
        return len(card) == 2 and card[0] in VALID_RANKS and card[1] in VALID_SUITS

    @staticmethod
    def evaluate_hand(hand):
        # First validate all cards
        for card in hand:
            if not HandValidator.is_valid_card(card):
                raise ValueError(f"Invalid card detected: {card}")
        
        values = sorted([rank_values[c[0]] for c in hand], reverse=True)
        suits = [c[1] for c in hand]
        
        # Check flush
        flush = len(set(suits)) == 1
        
        # Check straight
        unique = sorted(list(set(values)))
        straight = False
        for i in range(len(unique)-4):
            if unique[i] - unique[i+4] == 4:
                straight = True
                break
        if not straight and set(values) >= {14,2,3,4,5}:
            straight = True
            values = [5,4,3,2,1]  # Treat A-2-3-4-5 as 5-high
            
        # Count duplicates
        count = {}
        for v in values: count[v] = count.get(v, 0) + 1
        sorted_counts = sorted(count.items(), key=lambda x: (-x[1], -x[0]))
        
        if straight and flush: return (8, max(values))
        if any(c[1]==4 for c in sorted_counts): return (7, sorted_counts[0][0])
        if len(sorted_counts)>1 and sorted_counts[0][1]==3 and sorted_counts[1][1]==2:
            return (6, sorted_counts[0][0])
        if flush: return (5, values)
        if straight: return (4, max(values))
        if sorted_counts[0][1] == 3: return (3, sorted_counts[0][0], values)
        if len(sorted_counts)>1 and sorted_counts[0][1]==2 and sorted_counts[1][1]==2:
            return (2, sorted_counts[0][0], sorted_counts[1][0], values)
        if sorted_counts[0][1] == 2: return (1, sorted_counts[0][0], values)
        return (0, values)

def create_hand_matrix():
    # Configuration
    N_VARIANTS = 20         # Reduced for demonstration
    BASE_SIMS = 1000        
    EXTRA_SIMS = 10000      # Reduced for faster testing
    ALL_CARDS = [r+s for s in VALID_SUITS for r in VALID_RANKS]

    # Initialize matrix
    hands = {
        'Straight Flush': [['2H','3H','4H','5H','6H']],
        'Four of a Kind': [['2H','2D','2S','2C','3H']],
        'Full House': [['3H','3D','2S','2C','2H']],
        'Flush': [['2H','3H','5H','7H','9H']],
        'Straight': [['AH','2D','3S','4C','5H']],
        'Three of a Kind': [['2H','2D','2S','3C','4H']],
        'Two Pair': [['3H','3D','2S','2C','4H']],
        **{f'Pair {r}': [] for r in reversed(VALID_RANKS)},
        **{f'High {r}': [] for r in ['A','K','Q','J']},
        'High Other':[['8H','2D','3S','4C','5H']]
    }

    def generate_valid_hand(base_cards, validate_fn):
        attempts = 0
        while attempts < 1000:
            # Generate random cards
            remaining_cards = [c for c in ALL_CARDS if c not in base_cards]
            if len(remaining_cards) < (5 - len(base_cards)):
                attempts += 1
                continue
                
            hand = base_cards + random.sample(remaining_cards, 5 - len(base_cards))
            try:
                if validate_fn(hand):
                    return hand
            except:
                pass
            attempts += 1
        return None

    # Generate valid hands
    print("Generating valid hands...")
    for hand_type in tqdm(hands.keys()):
        if hand_type in ['Straight Flush', 'Four of a Kind', 'Full House',
                        'Flush', 'Straight', 'Three of a Kind', 'Two Pair']:
            continue  # Use predefined examples
            
        if 'Pair' in hand_type:
            pair_rank = hand_type.split()[-1]
            base_cards = [f"{pair_rank}{s}" for s in random.sample(VALID_SUITS, 2)]
            validate_fn = lambda h: len({c[0] for c in h}) == 4  # Exactly 1 pair
        else:  # High cards
            high_rank = hand_type.split()[-1]
            base_cards = [f"{high_rank}{random.choice(VALID_SUITS)}"]
            validate_fn = lambda h: HandValidator.evaluate_hand(h)[0] == 0  # Pure high card
            
        variants = []
        for _ in range(N_VARIANTS):
            hand = generate_valid_hand(base_cards, validate_fn)
            if hand:
                variants.append(hand)
        hands[hand_type] = variants[:N_VARIANTS]

    # Simulation logic
    print("\nRunning simulations...")
    matrix = pd.DataFrame(index=hands.keys(), columns=VALID_RANKS)
    
    for hand_type in tqdm(hands.keys()):
        for dealer_rank in VALID_RANKS:
            total_wins = 0
            total_sims = 0
            
            for variant in hands[hand_type]:
                excluded = variant + [f"{dealer_rank}H"]
                deck = [c for c in ALL_CARDS if c not in excluded]
                
                if len(deck) < 4:
                    continue
                    
                # Initial simulations
                wins = 0
                for _ in range(BASE_SIMS):
                    dealer_hand = [f"{dealer_rank}H"] + random.sample(deck, 4)
                    try:
                        player_score = HandValidator.evaluate_hand(variant)
                        dealer_score = HandValidator.evaluate_hand(dealer_hand)
                        wins += 1 if player_score > dealer_score else 0
                    except:
                        continue
                
                # Precision boost
                p = wins/BASE_SIMS if BASE_SIMS > 0 else 0
                if 0.45 <= p <= 0.6 and len(deck) >= 4:
                    for _ in range(EXTRA_SIMS):
                        dealer_hand = [f"{dealer_rank}H"] + random.sample(deck, 4)
                        try:
                            player_score = HandValidator.evaluate_hand(variant)
                            dealer_score = HandValidator.evaluate_hand(dealer_hand)
                            wins += 1 if player_score > dealer_score else 0
                        except:
                            continue
                    total_sims += BASE_SIMS + EXTRA_SIMS
                else:
                    total_sims += BASE_SIMS
                
                total_wins += wins
                
            if total_sims > 0:
                matrix.loc[hand_type, dealer_rank] = round(total_wins/total_sims, 2)
            else:
                matrix.loc[hand_type, dealer_rank] = 0.0

    # Visualization
    plt.figure(figsize=(16, 10))
    sns.heatmap(
        matrix.astype(float),
        annot=True,
        cmap=sns.diverging_palette(220, 20, as_cmap=True),
        vmin=0.3,
        vmax=0.7,
        center=0.5,
        fmt='.2f',
        linewidths=0.5,
        cbar_kws={'label': 'Win Probability'}
    )
    plt.title("Poker Hand Matrix (Validated Hands)", pad=20)
    plt.xlabel("Dealer Up Card", labelpad=15)
    plt.ylabel("Player Hand Type", labelpad=15)
    plt.tight_layout()
    plt.show()
    
    return matrix

# Generate and display matrix
hand_matrix7 = create_hand_matrix()


Generating valid hands...


100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 138.24it/s]



Running simulations...


 56%|█████████████████████████████████████████████▉                                    | 14/25 [00:51<01:16,  6.98s/it]

In [ ]:
hand_matrix7.to_csv

In [ ]:
hand_matrix7.to_csv('hand_matrix7.csv', index=True)
